In [86]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from census import Census
import gmaps

In [2]:
# Converting CSV to DataFrame
file="csv_data/HCAHPS - Hospital.csv"
file2="csv_data/Hospital General Information.csv"
file3="csv_data/Structural Measures - Hospital.csv"
hospital=pd.read_csv(file) #contain hospital ID, name, and address
hospital_Gen_Info=pd.read_csv(file2) # contain hospital type and and ownership
hospital_Efiles=pd.read_csv(file3) #contain hospital and whether it has electronic lab results capabilities

In [3]:
hospital.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW


In [4]:
hospital_Gen_Info.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes
1,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,Yes
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,Acute Care Hospitals,Government - Hospital District or Authority,Yes
3,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,Yes
4,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,Yes


In [5]:
hospital_Efiles.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Measure Name,Measure ID,Measure Response
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Able to receive lab results electronically,OP_12,Yes
1,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,"Able to track patients' lab results, tests, an...",OP_17,No
2,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Able to receive lab results electronically,OP_12,Yes
3,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,"Able to track patients' lab results, tests, an...",OP_17,Yes
4,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,Able to receive lab results electronically,OP_12,Yes


In [6]:
# coding: utf-8

# In[2]:
import pandas as pd
import numpy as np
# In[3]:
zip_data = pd.read_csv("zip_codes_states.csv")
# In[4]:
zip_data.head()
# In[5]:
zip_codes = []
for z in zip_data["zip"]:
    if len(str(z))==3:
        z = "00"+str(z)
    else:
        if len(str(z))==4:
            z = "0"+str(z)
        else:
            z = str(z)
    zip_codes.append(z)
zip_data["zip_code"]=zip_codes
# In[6]:
zip_data = zip_data.drop(["zip"],axis=1)
zip_data = zip_data.rename(columns={'zip_code':'ZIP Code'})
zip_data['ZIP Code'].astype(str)
zip_data.head()

,ZIP Code,latitude,longitude,city,state,county
0,00501,40.922326,-72.637078,Holtsville,NY,Suffolk
1,00544,40.922326,-72.637078,Holtsville,NY,Suffolk
2,00601,18.165273,-66.722583,Adjuntas,PR,Adjuntas
3,00602,18.393103,-67.180953,Aguada,PR,Aguada
4,00603,18.455913,-67.145780,Aguadilla,PR,Aguadilla


In [7]:
# In[7]:
claim_data = pd.read_csv("Medicare Hospital Spending by Claim.csv")
# In[8]:
claim_data = claim_data.sort_values("Provider_ID")
df = pd.DataFrame(claim_data.loc[claim_data['Claim_Type']=="Total"])
df = df.drop(["Percent_of_Spending_Hospital","Percent_of_Spending_State","Percent_of_Spending_Nation","Period","Claim_Type"],axis=1)
df.to_csv("hospital_claim_data",sep="\t")
# In[9]:
df = df.rename(columns={'Hospital_Name':'Hospital Name'})
df.head()

,Hospital Name,Provider_ID,State,Avg_Spending_Per_Episode_Hospital,Avg_Spending_Per_Episode_State,Avg_Spending_Per_Episode_Nation,Start_Date,End_Date
17083,SOUTHEAST ALABAMA MEDICAL CENTER,10001,AL,20711,19139,20302,1012016,12312016
56338,MARSHALL MEDICAL CENTERS,10005,AL,16849,19139,20302,1012016,12312016
21475,ELIZA COFFEE MEMORIAL HOSPITAL,10006,AL,19553,19139,20302,1012016,12312016
17084,MIZELL MEMORIAL HOSPITAL,10007,AL,15899,19139,20302,1012016,12312016
65059,CRENSHAW COMMUNITY HOSPITAL,10008,AL,12254,19139,20302,1012016,12312016


In [8]:
master = hospital_Gen_Info.merge(df,on='Hospital Name',how="inner")


In [9]:
master = master.drop(['State_y'],axis=1)


In [10]:
master = master.rename(columns={'State_x':'State'})
master.head()

,Provider ID,Hospital Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services,Provider_ID,Avg_Spending_Per_Episode_Hospital,Avg_Spending_Per_Episode_State,Avg_Spending_Per_Episode_Nation,Start_Date,End_Date
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes,10001,20711,19139,20302,1012016,12312016
1,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,LAUDERDALE,Acute Care Hospitals,Government - Hospital District or Authority,Yes,10006,19553,19139,20302,1012016,12312016
2,10007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,Yes,10007,15899,19139,20302,1012016,12312016
3,10008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,Yes,10008,12254,19139,20302,1012016,12312016
4,10012,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,DE KALB,Acute Care Hospitals,Proprietary,Yes,10012,15112,19139,20302,1012016,12312016


In [15]:
election_data = pd.read_csv("csv_data/election_data.csv",header=1)

In [31]:
election_data['political score']
dem_score = -(election_data['d']+election_data['d.1']+election_data['d.2']+election_data['d.3']+election_data['d.4'])/5
rep_score = (election_data['r']+election_data['r.1']+election_data['r.2']+election_data['r.3']+election_data['r.4'])/5
election_data['political score']=(dem_score + rep_score)
election_data.head()

,State,State Abbrv,d,r,d.1,r.1,d.2,r.2,d.3,r.3,d.4,r.4,political score
0,Nationwide,NaN,48.0,45.9,51.0,47.1,52.9,45.6,48.3,50.7,48.4,47.9,-2.28
1,Alabama,AL,34.4,62.1,38.4,60.5,38.7,60.3,36.8,62.5,41.6,56.5,22.40
2,Alaska,AK,36.6,51.3,40.8,54.8,37.9,59.4,35.5,61.1,27.7,58.6,21.34
3,Arizona,AZ,44.6,48.1,44.4,53.5,44.9,53.4,44.3,54.8,44.7,51.0,7.58
4,Arkansas,AR,33.7,60.6,36.9,60.6,38.9,58.7,44.5,54.3,45.9,51.3,17.12


In [71]:
poli_score_df = election_data.drop(columns=['State','d','r','d.1','r.1','d.2','r.2',
                                           'd.3','r.3','d.4','r.4'])
poli_score_df = poli_score_df.dropna()
poli_score_df = poli_score_df.rename(columns={'State Abbrv':'State'})
poli_score_df = poli_score_df.set_index('State')
poli_score_df.head()

,political score
State,
AL,22.40
AK,21.34
AZ,7.58
AR,17.12
CA,-19.74


In [77]:
hospital_by_state = pd.DataFrame(master['State'].value_counts())
hospital_by_state.head()
poli_health_df = poli_score_df.join(hospital_by_state)
poli_health_df = poli_health_df.rename(columns={'State':'Hospitals'})
poli_health_df.head()

,political score,Hospitals
State,,
AL,22.40,72
AK,21.34,8
AZ,7.58,59
AR,17.12,42
CA,-19.74,314
